load data

In [1]:
import pandas as pd

In [2]:
he = pd.read_csv('data/HE.csv', encoding='utf-8', sep=";", na_filter=False)

In [3]:
he = he.loc[:, 'Country': "ICT"][he.Website != ""]
he.head()

,Country,University,Funding,Website,Majors,Rated,ARWU 2020,THE 2021,QS 2021,ICT
0,Bahrain,Ahlia University,Private,http://www.ahliauniversity.edu.bh,"'business administration', 'accountancy', 'mod...",False,,,,True
1,Bahrain,AMA International University - Bahrain,Private,http://www.amaiu.edu.bh,"'business administration', 'business informati...",False,,,,True
2,Bahrain,American University of Bahrain,Private,www.aubh.edu.bh,"'business and management', 'multimedia design'...",False,,,,True
3,Bahrain,Applied Science University,Private,http://www.asu.edu.bh,"'business administration', 'law', 'information...",True,,,651-700,True
4,Bahrain,Arab Open University - Bahrain Branch,Private,http://www.aou.org.bh,"'business administration', 'information techno...",False,,,,True


connecting to db

In [4]:
import mysql.connector

In [5]:
con = mysql.connector.connect(
    host='127.0.0.1',
    port=33062,
    database="edu_orgs", user='exam_db', password='NarrowBezel3')

In [6]:
cur=con.cursor(dictionary=True)

Test connection

In [7]:
cur.execute("SELECT * FROM institutions LIMIT 0, 5")

In [8]:
data=cur.fetchall()
data

[{'institution_id': 1,
  'institution_name': 'National Research University Higher School of Economics',
  'institution_short': 'HSE',
  'country_id': 11,
  'website': 'https://www.hse.ru/en/',
  'funding': 'public'},
 {'institution_id': 2,
  'institution_name': 'Tomsk State Pedagogical Univesity',
  'institution_short': 'TSPU',
  'country_id': 11,
  'website': 'https://www.tspu.edu.ru/en/',
  'funding': 'public'}]

## Check what is in the db

geographic data

In [9]:
cur.execute("""
    SELECT country_name, region_name FROM countries
    JOIN geo_regions USING(region_id)
    WHERE region_id IN (2,3,4);
""")

In [10]:
data=cur.fetchall()
data

[{'country_name': 'Bahrain', 'region_name': 'Arab States'},
 {'country_name': 'Iraq', 'region_name': 'Arab States'},
 {'country_name': 'Jordan', 'region_name': 'Arab States'},
 {'country_name': 'Kuwait', 'region_name': 'Arab States'},
 {'country_name': 'Lebanon', 'region_name': 'Arab States'},
 {'country_name': 'Oman', 'region_name': 'Arab States'},
 {'country_name': 'Qatar', 'region_name': 'Arab States'},
 {'country_name': 'Saudi Arabia', 'region_name': 'Arab States'},
 {'country_name': 'United Arab Emirates', 'region_name': 'Arab States'},
 {'country_name': 'Pakistan', 'region_name': 'Asia and the Pacific'},
 {'country_name': 'Russian Federation',
  'region_name': 'Europe and North America'}]

Filter institutions by country

In [11]:
cur.execute("""
    SELECT country_acronym as country,
    institution_short as university,
    website, funding
    FROM institutions JOIN countries USING(country_id)
    WHERE country_id=11;
""")

In [12]:
data=cur.fetchall()
data

[{'country': 'RU',
  'university': 'HSE',
  'website': 'https://www.hse.ru/en/',
  'funding': 'public'},
 {'country': 'RU',
  'university': 'TSPU',
  'website': 'https://www.tspu.edu.ru/en/',
  'funding': 'public'}]

Check the category of the institution: HE or TVET (TVET to be collected)

In [13]:
cur.execute("""
    SELECT country_acronym as country, institution_short as university,
    type_full as category, website FROM
        (SELECT institution_id, country_id, institution_short, website, type_full FROM institutions 
        JOIN institution_type USING(institution_id)) as tbl1
    JOIN countries USING(country_id);
""")

In [14]:
data=cur.fetchall()
data

[{'country': 'RU',
  'university': 'HSE',
  'category': 'Higher Education',
  'website': 'https://www.hse.ru/en/'},
 {'country': 'RU',
  'university': 'TSPU',
  'category': 'Technical and Vocational Education and Training',
  'website': 'https://www.tspu.edu.ru/en/'},
 {'country': 'RU',
  'university': 'TSPU',
  'category': 'Higher Education',
  'website': 'https://www.tspu.edu.ru/en/'}]

Check if it is listed in international ratings and has IT courses

In [15]:
cur.execute("""
    SELECT * FROM
        (SELECT institution_id, 
        institution_short as university,
        arwu, the, qs
        FROM institutions JOIN ratings USING(institution_id)) as tbl1
    JOIN 
        (SELECT institution_id, 
        GROUP_CONCAT(DISTINCT major SEPARATOR ", ") as majors 
        FROM majors WHERE major REGEXP 
        '(computer science)|(artificial intelligence)|(data science)'
        GROUP BY institution_id) as tbl2
    USING(institution_id);
""")

In [16]:
data=cur.fetchall()
data

[{'institution_id': 1,
  'university': 'HSE',
  'arwu': '801-900',
  'the': '251-300',
  'qs': '298',
  'majors': 'computer science, data science'}]

## Populate database

transform pandas into to json

In [17]:
import json

In [18]:
cur.execute("""
    SELECT country_name as Country, country_id as id FROM countries;
""")
countries=cur.fetchall()
countries = {entry['Country']: entry['id'] for entry in countries}
countries

{'Bahrain': 1,
 'Iraq': 2,
 'Jordan': 3,
 'Kuwait': 4,
 'Lebanon': 5,
 'Oman': 6,
 'Pakistan': 10,
 'Qatar': 7,
 'Russian Federation': 11,
 'Saudi Arabia': 8,
 'United Arab Emirates': 9}

In [19]:
result = he.to_json(orient="records")
unis = json.loads(result)
json.dumps(unis[9])

'{"Country": "Bahrain", "University": "Gulf University Al-Jame`a Al-Khaleejia", "Funding": "Public", "Website": "http://www.gulfuniversity.edu.bh", "Majors": "\'administrative and financial sciences\', \'interior design engineering\', \'network administration\', \'web and mobile applications development\'", "Rated": false, "ARWU 2020": "", "THE 2021": "", "QS 2021": "", "ICT": true}'

In [20]:
len(unis)

555

In [21]:
for a in unis:
    name = a["University"]
    if len(name) > 105: # filter some of the not accredited
        print(name)

AlKhair University (Admission has been banned at all levels Bachelors & Masters w.e.f Fall 2016 & MS/MPhil/PhD w.e.f Fall 2014)
Dadabhoy Institute of Higher Education (Admissions allowed only at SNPA-17/B Block 3 P.E.C.H. Society,Shaheed-e-Millat Road Karachi)
Sindh Institute of Management & Technology (Admissions allowed only at LS 37/10 Sector-15 Main Korangi Industrial Karachi in specific departements Maximum intake 40 till December 2020)


populate `insitutions` table using packages

In [22]:
%%time
data=[]
for a in unis:
    name = a["University"]
    if len(name) > 105: # filter not accredited
        continue
    country_id = countries[a['Country']]
    website = a["Website"]
    funding = a["Funding"].lower()
    data.append("('{}', {}, '{}', '{}')".format(name, country_id, website, funding))
    
for i in range(6):
    req="INSERT INTO institutions (institution_name, country_id, website, funding) VALUES " + \
    ",".join(data[i*100:(i+1)*100])
    cur.execute(req)

    con.commit()

CPU times: user 1.72 ms, sys: 2.51 ms, total: 4.23 ms
Wall time: 90.5 ms


In [23]:
cur.execute("""
    SELECT institution_name as name, institution_short as acronym FROM institutions LIMIT 0,10;
""")
data=cur.fetchall()
data

[{'name': 'National Research University Higher School of Economics',
  'acronym': 'HSE'},
 {'name': 'Tomsk State Pedagogical Univesity', 'acronym': 'TSPU'},
 {'name': 'Ahlia University', 'acronym': None},
 {'name': 'AMA International University - Bahrain', 'acronym': None},
 {'name': 'American University of Bahrain', 'acronym': None},
 {'name': 'Applied Science University', 'acronym': None},
 {'name': 'Arab Open University - Bahrain Branch', 'acronym': None},
 {'name': 'Bahrain Institute of Banking and Finance', 'acronym': None},
 {'name': 'Arabian Gulf University', 'acronym': None},
 {'name': 'Bahrain Polytechnic', 'acronym': None}]

update `institute_short` column

In [24]:
cur.execute("""
UPDATE institutions 
SET institution_short = UPPER(REPLACE(REGEXP_SUBSTR(website, "[a-z]+\.edu"), "\.edu", ""))
    WHERE institution_id IN ( 
        SELECT institution_id FROM 
        (SELECT institution_id, website FROM institutions
        WHERE website REGEXP "\.edu\." and LENGTH(website) < 25) as tbl1);
"""
)

In [25]:
cur.execute("""
    SELECT institution_name as name, institution_short as acronym FROM institutions
    WHERE institution_short <> '' LIMIT 0, 10;
""")
data = cur.fetchall()
data

[{'name': 'National Research University Higher School of Economics',
  'acronym': 'HSE'},
 {'name': 'Tomsk State Pedagogical Univesity', 'acronym': 'TSPU'},
 {'name': 'AMA International University - Bahrain', 'acronym': 'AMAIU'},
 {'name': 'American University of Bahrain', 'acronym': 'AUBH'},
 {'name': 'Applied Science University', 'acronym': 'ASU'},
 {'name': 'Arabian Gulf University', 'acronym': 'AGU'},
 {'name': 'Royal University for Women', 'acronym': 'RUW'},
 {'name': 'The Kingdom University', 'acronym': 'KU'},
 {'name': 'University College of Bahrain', 'acronym': 'UCB'},
 {'name': 'Al-Mansour University College', 'acronym': 'MUC'}]

populate `ratings` table

In [26]:
cur.execute("""
    SELECT institution_name as University, institution_id as id FROM institutions;
""")
uni_ids=cur.fetchall()
uni_ids = {entry["University"]: entry['id'] for entry in uni_ids}

len(uni_ids)

547

In [27]:
%%time

for a in unis:
    if a['Rated'] != True:
        continue
    uni_id = uni_ids[a["University"]]
    arwu = a["ARWU 2020"]
    the = a["THE 2021"]
    qs = a["QS 2021"]
    try:
        sss = f"""
        INSERT INTO ratings (institution_id, arwu, the, qs) 
        VALUES ({uni_id}, '{arwu}', '{the}', '{qs}')"""
        cur.execute(sss)
    except:
        print(
            """Error - INSERT INTO ratings (institution_id, arwu, the, qs)
            VALUES ('"+uni_id+"','"+arwu+"', '"+the+"', '"+qs+"')""")

    con.commit()

CPU times: user 16.1 ms, sys: 5 ms, total: 21.1 ms
Wall time: 519 ms


In [28]:
cur.execute("""
    SELECT country_name as country, institution_short as university,
    arwu, the, qs
    FROM (SELECT * FROM institutions JOIN countries USING(country_id)) as tbl1
    JOIN ratings USING(institution_id) WHERE institution_id <> 1 LIMIT 0,5 
""")
data = cur.fetchall()
data

[{'country': 'Jordan',
  'university': 'ASU',
  'arwu': '',
  'the': '',
  'qs': '651-700'},
 {'country': 'Saudi Arabia',
  'university': None,
  'arwu': '101-150',
  'the': '201-250',
  'qs': '143'},
 {'country': 'Saudi Arabia',
  'university': None,
  'arwu': '401-500',
  'the': '501-600',
  'qs': '186'},
 {'country': 'Lebanon',
  'university': 'AUB',
  'arwu': '601-700',
  'the': '301-350',
  'qs': '220'},
 {'country': 'Iraq', 'university': None, 'arwu': '', 'the': '1001+', 'qs': ''}]

populate `institution_type` table: if comes from HE.csv -> HE

In [29]:
cur.execute("""
    SELECT institution_id as idx FROM institutions;
""")
uni_ids=cur.fetchall()

In [30]:
%%time
data=[]
for entry in uni_ids:
    if entry['idx'] < 3:
        continue
    type_full = 'Higher Education'
    type_short = 'HE'
    data.append("({}, '{}', '{}')".format(entry['idx'], type_full, type_short))

for i in range(6):
    req="INSERT INTO institution_type (institution_id, type_full, type_short) VALUES " + \
    ",".join(data[i*100:(i+1)*100])
    cur.execute(req)

    con.commit()

CPU times: user 2.81 ms, sys: 1.72 ms, total: 4.53 ms
Wall time: 96.7 ms


In [32]:
cur.execute("""
    SELECT * FROM institution_type LIMIT 0,5 
""")
data = cur.fetchall()
data

[{'level_id': 1,
  'institution_id': '1',
  'type_full': 'Higher Education',
  'type_short': 'HE'},
 {'level_id': 3,
  'institution_id': '2',
  'type_full': 'Higher Education',
  'type_short': 'HE'},
 {'level_id': 4,
  'institution_id': '2',
  'type_full': 'Technical and Vocational Education and Training',
  'type_short': 'TVET'},
 {'level_id': 2353,
  'institution_id': '9182',
  'type_full': 'Higher Education',
  'type_short': 'HE'},
 {'level_id': 2354,
  'institution_id': '9183',
  'type_full': 'Higher Education',
  'type_short': 'HE'}]

Populate `majors` table

In [33]:
import re

In [34]:
cur.execute("""
    SELECT major, degree, institution_id FROM majors 
""")
data = cur.fetchall()
data

[{'major': 'computer science', 'degree': 'Master', 'institution_id': 1},
 {'major': 'computer science', 'degree': 'Doctoral', 'institution_id': 1},
 {'major': 'computer science', 'degree': 'Bachelor', 'institution_id': 1},
 {'major': 'world economy', 'degree': 'Bachelor', 'institution_id': 1},
 {'major': 'world economy', 'degree': 'Master', 'institution_id': 1},
 {'major': 'world economy', 'degree': 'Doctoral', 'institution_id': 1},
 {'major': 'business informatics', 'degree': 'Bachelor', 'institution_id': 1},
 {'major': 'data science', 'degree': 'Master', 'institution_id': 1},
 {'major': 'psychology', 'degree': 'Bachelor', 'institution_id': 2},
 {'major': 'primary school teaching',
  'degree': 'Bachelor',
  'institution_id': 2},
 {'major': 'translation strudies', 'degree': 'Doctoral', 'institution_id': 2},
 {'major': 'language teacher', 'degree': 'Certificate', 'institution_id': 2}]

In [35]:
website2majors = {entry["Website"]: entry["Majors"].lower().replace("'", "") for entry in unis if entry['Majors'] != ""}
len(website2majors)

476

In [37]:
for website, majors in website2majors.items():
    print(website, majors)
    break

http://www.ahliauniversity.edu.bh business administration, accountancy, modern languages, management, information management, finance, public relations, mass communication, arabic, interior design, mathematics, telecommunications engineering, physical therapy, multimedia systems, marketing, economics, information technology and computer science, graphic design, computer engineering, mobile and network engineering, computer and communication engineering


In [38]:
cur.execute("""
    SELECT website as website, institution_id as id FROM institutions;
""")
website2id=cur.fetchall()
website2id = {entry["website"]: entry['id'] for entry in website2id}

len(website2id)

554

In [39]:
data=[]

for website, majors in website2majors.items():
    majors = majors.strip().split(", ")
    if website in website2id:
        idx = website2id[website]
        
    for major in majors:
        re.sub('[\?\.,]', "", major)
        if len(major) < 100:
            data.append("('{}', {})".format(major, idx))
    
print(data[100:110])

["('operations engineering (instrumentation and control)', 9190)", "('computing and systems development', 9190)", "('computing and systems development', 9190)", "('business administration,', 9190)", "('administrative and financial sciences', 9191)", "('interior design engineering', 9191)", "('network administration', 9191)", "('web and mobile applications development', 9191)", "('faculty of instruction', 9192)", "('faculty of arts and humanities', 9192)"]


In [40]:
data = [entry for entry in data if entry] # somehow there invalid entries
len(data)

10877

In [41]:
%%time
top = int(len(data) / 100) + 1

for i in range(top):
    req="INSERT INTO majors (major, institution_id) VALUES " + \
    ",".join(data[i*100:(i+1)*100])
    cur.execute(req)

    con.commit()

CPU times: user 19.5 ms, sys: 21.2 ms, total: 40.7 ms
Wall time: 1.27 s


In [42]:
cur.execute("""
    SELECT major, degree, institution_id FROM majors WHERE major_id >12 LIMIT 0,20 
""")
data = cur.fetchall()
data
# later it is possible to update degrees using major (some of them have degrees in the name of a major)

[{'major': 'business administration', 'degree': None, 'institution_id': 9182},
 {'major': 'accountancy', 'degree': None, 'institution_id': 9182},
 {'major': 'modern languages', 'degree': None, 'institution_id': 9182},
 {'major': 'management', 'degree': None, 'institution_id': 9182},
 {'major': 'information management', 'degree': None, 'institution_id': 9182},
 {'major': 'finance', 'degree': None, 'institution_id': 9182},
 {'major': 'public relations', 'degree': None, 'institution_id': 9182},
 {'major': 'mass communication', 'degree': None, 'institution_id': 9182},
 {'major': 'arabic', 'degree': None, 'institution_id': 9182},
 {'major': 'interior design', 'degree': None, 'institution_id': 9182},
 {'major': 'mathematics', 'degree': None, 'institution_id': 9182},
 {'major': 'telecommunications engineering',
  'degree': None,
  'institution_id': 9182},
 {'major': 'physical therapy', 'degree': None, 'institution_id': 9182},
 {'major': 'multimedia systems', 'degree': None, 'institution_id': 

create table `ict_education` to store info if the university is rated and has ict majors (e.g. computer science TBD)

In [43]:
cur.execute("DROP TABLE IF EXISTS ict_education")

In [44]:
cur.execute("""
    CREATE TABLE `ict_education` (
    `entry_id` INT NOT NULL AUTO_INCREMENT,
    `institution_id` INT NOT NULL,
    `is_rated` TINYINT NULL,
    `is_ict` TINYINT NULL,
    PRIMARY KEY (`entry_id`),
    UNIQUE INDEX `insitution_id_UNIQUE` (`institution_id` ASC) VISIBLE);
""")

con.commit()

get ids of stored unis

In [45]:
cur.execute("""
    SELECT institution_id as idx FROM institutions;
""")
uni_ids=cur.fetchall()
uni_ids = [entry["idx"] for entry in uni_ids]

len(uni_ids)

554

In [46]:
%%time

for idx in uni_ids:
    try:
        sss = f"""
        INSERT INTO ict_education (institution_id, is_rated, is_ict) 
        VALUES ({idx}, {0}, {0})"""
        cur.execute(sss)
    except:
        print(
            """Error - INSERT INTO ict_education (institution_id) 
        VALUES ({uni_id})""")

con.commit()

CPU times: user 37.3 ms, sys: 28.9 ms, total: 66.2 ms
Wall time: 224 ms


In [47]:
cur.execute("""
    UPDATE ict_education SET is_ICT = 1
    WHERE institution_id IN 
        (SELECT institution_id FROM 
            (SELECT institution_id, GROUP_CONCAT(major) as majors
                FROM (SELECT DISTINCT institution_id, major FROM majors
                    WHERE major 
                        REGEXP '(computer science)|(artificial intelligence)|(data science)') as tbl1
            GROUP BY institution_id) as tbl2);
""")

In [48]:
cur.execute("""
    UPDATE ict_education SET is_rated = 1
    WHERE institution_id IN 
        (SELECT institution_id FROM (SELECT * FROM ratings) as tbl1);
""")

In [49]:
cur.execute("""
    SELECT * FROM ict_education LIMIT 0,20 
""")
data = cur.fetchall()
data 

[{'entry_id': 1, 'institution_id': 1, 'is_rated': 1, 'is_ict': 1},
 {'entry_id': 2, 'institution_id': 2, 'is_rated': 0, 'is_ict': 0},
 {'entry_id': 3, 'institution_id': 9182, 'is_rated': 0, 'is_ict': 1},
 {'entry_id': 4, 'institution_id': 9183, 'is_rated': 0, 'is_ict': 1},
 {'entry_id': 5, 'institution_id': 9184, 'is_rated': 0, 'is_ict': 1},
 {'entry_id': 6, 'institution_id': 9185, 'is_rated': 0, 'is_ict': 1},
 {'entry_id': 7, 'institution_id': 9186, 'is_rated': 0, 'is_ict': 0},
 {'entry_id': 8, 'institution_id': 9187, 'is_rated': 0, 'is_ict': 0},
 {'entry_id': 9, 'institution_id': 9188, 'is_rated': 0, 'is_ict': 0},
 {'entry_id': 10, 'institution_id': 9189, 'is_rated': 0, 'is_ict': 0},
 {'entry_id': 11, 'institution_id': 9190, 'is_rated': 0, 'is_ict': 0},
 {'entry_id': 12, 'institution_id': 9191, 'is_rated': 0, 'is_ict': 0},
 {'entry_id': 13, 'institution_id': 9192, 'is_rated': 1, 'is_ict': 0},
 {'entry_id': 14, 'institution_id': 9193, 'is_rated': 0, 'is_ict': 0},
 {'entry_id': 15, 'in